<a href="https://colab.research.google.com/github/prantik-pdeb/GSoC2022-QML/blob/main/Task_3_(b).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task III:

# 3-(b) Implementing Quantum Neural Network (QNN) using Tensorflow Quantum 

The electron-photon dataset (which can be found [here](https://www.google.com/url?q=https%3A%2F%2Fgithub.com%2FML4SCI%2FML4SCI_GSoC%2Ftree%2Fmain%2FQMLHEP%2Fqcnn&sa=D&source=docs)) contains 100 samples for training and another 100 for testing, laid out as follows:

a. data["x_train"]: Training dataset of 100 32x32 images containing the particles' energy (100, 32, 32)

b. data["y_train"]:" Training labels, 0 = "photon", 1 = "electron" (100,)

c. data["x_test"]: Test dataset of 100 32x32 images containing the particles' energy (100, 32, 32)

d. data["y_test"]:" Test labels, 0 = "photon", 1 = "electron" (100,)

In [ ]:
# downloading tensorflow quantum
!pip install tensorflow-quantum

In [ ]:
#importing tensorflow and module dependecies 
import numpy as np
import tensorflow as tf
import importlib, pkg_resources
importlib.reload(pkg_resources)
import tensorflow_quantum as tfq
import collections
import operator
import cirq
import sympy

#importing visualozation tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit
import matplotlib.image  as mpimg

In [ ]:
'''loading data electron-photon dataset containts 100 samples
for training and 100 sampes for test'''
'''['x_train'] and ['x_test'] training and test dataset of 100 32*32 images 
['y_train'] and ['y_test'] training and test level with 0 = photon, 1 = electron'''

dataset = np.load('/content/electron-photon.npz')

In [ ]:
with dataset as data:
  x_train = data['x_train']
  y_train = data['y_train']
  x_test = data['x_test']
  y_test= data['y_test']

#Data Visualization

In [ ]:
plt.imshow(x_train[1])
plt.colorbar()

In [ ]:
plt.imshow(x_train[51])
plt.colorbar()

In [ ]:
plt.imshow(x_test[1])
plt.colorbar()

In [ ]:
plt.imshow(x_test[51])
plt.colorbar()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [ ]:
BATCH_SIZE = 8
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Classical Neural Network 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32, 32)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history= model.fit(train_dataset, epochs=100, validation_data=test_dataset)

In [ ]:
train_loss, train_acc = model.evaluate(train_dataset)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

# Result: Visualization of Classical Neural Network

In [ ]:
train_acc=history.history['accuracy']
test_acc=history.history['val_accuracy']
train_loss=history.history['loss']
test_loss=history.history['val_loss']

epochs=range(len(train_acc)) 
plt.plot(epochs, train_acc, 'r', label= "Training Accuracy")
plt.plot(epochs, test_acc, 'b', label = "Test Accuracy")
plt.title('Training and testing accuracy')
plt.figure()

plt.plot(epochs, train_loss, 'r',label= "Training Loss")
plt.plot(epochs, test_loss, 'b', label="Test Loss")
plt.title('Training and testing loss')
plt.figure()




---

# Quantum Neural Network (QNN)



In [ ]:
# Resize image  
def crop_img(img, dimension):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, dimension))
    end = tuple(map(operator.add, start, dimension))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [ ]:
crop_size = (16, 16)
x_train_cropped_size = np.array([crop_img(i, crop_size) for i in x_train])
x_test_cropped_size = np.array([crop_img(i, crop_size) for i in x_test])

In [ ]:
print('Shape of x_train dataset:',x_train_cropped_size.shape)
print('Shape of x_test dataset:',x_test_cropped_size.shape)

In [ ]:
plt.imshow(x_train_cropped_size[1])
plt.colorbar()

In [ ]:
plt.imshow(x_test_cropped_size[1])
plt.colorbar()

In [ ]:
# Add the color channel into the dataset (batch_size, height, width, channel)
x_train_new_size = np.reshape(x_train_cropped_size, list(x_train_cropped_size.shape)+[1])
x_test_new_size = np.reshape(x_test_cropped_size, list(x_test_cropped_size.shape)+[1])

In [ ]:
print('Shape of x_train dataset:',x_train_new_size.shape)
print('Shape of x_test dataset:',x_test_new_size.shape)

In [ ]:
#An image size of 32x32 is much too large for current quantum computers. Resize the image down to 4x4
# Using the image resize function from tensorflow library for tf.image.resize
x_train_small = np.array([tf.image.resize(img, (4,4)).numpy() for img in x_train_new_size])
x_test_small = np.array([tf.image.resize(img, (4,4)).numpy() for img in x_test_new_size])
x_train_small = np.reshape(x_train_small, x_train_small.shape[:3])
x_test_small = np.reshape(x_test_small, x_test_small.shape[:3])

In [ ]:
def convert_to_circuit(image):
    """Encode truncated classical image into quantum datapoint."""
    values = np.ndarray.flatten(image)
    qubits = cirq.GridQubit.rect(4, 4)
    circuit = cirq.Circuit()
    for i, value in enumerate(values):
        if value:
            circuit.append(cirq.X(qubits[i]))
    return circuit

In [ ]:
x_train_circ = [convert_to_circuit(x) for x in x_train_small]
x_test_circ = [convert_to_circuit(x) for x in x_test_small]

In [ ]:
SVGCircuit(x_train_circ[0])

In [ ]:
bin_img = x_train_small[0]
indices = np.array(np.where(bin_img)).T
indices

In [ ]:
x_train_tfcirc = tfq.convert_to_tensor(x_train_circ)
x_test_tfcirc = tfq.convert_to_tensor(x_test_circ)

In [ ]:
class CircuitLayerBuilder():
    def __init__(self, data_qubits, readout):
        self.data_qubits = data_qubits
        self.readout = readout

    def add_layer(self, circuit, gate, prefix):
        for i, qubit in enumerate(self.data_qubits):
            symbol = sympy.Symbol(prefix + '-' + str(i))
            circuit.append(gate(qubit, self.readout)**symbol)

In [ ]:
demo_builder = CircuitLayerBuilder(data_qubits = cirq.GridQubit.rect(4,1),
                                   readout=cirq.GridQubit(-1,-1))

circuit = cirq.Circuit()
demo_builder.add_layer(circuit, gate = cirq.XX, prefix='xx')
SVGCircuit(circuit)

In [ ]:
def create_quantum_model():
    """Create a QNN model circuit and readout operation to go along with it."""
    data_qubits = cirq.GridQubit.rect(4, 4)  # a 4x4 grid.
    readout = cirq.GridQubit(-1, -1)         # a single qubit at [-1,-1]
    circuit = cirq.Circuit()

    # Prepare the readout qubit.
    circuit.append(cirq.X(readout))
    circuit.append(cirq.H(readout))

    builder = CircuitLayerBuilder(
        data_qubits = data_qubits,
        readout=readout)

    # Then add layers (experiment by adding more).
    builder.add_layer(circuit, cirq.XX, "xx1")
    builder.add_layer(circuit, cirq.ZZ, "zz1")

    # Finally, prepare the readout qubit.
    circuit.append(cirq.H(readout))

    return circuit, cirq.Z(readout)

In [ ]:
model_circuit, model_readout = create_quantum_model()

In [ ]:
# Build the Keras model.
model = tf.keras.Sequential([
    # The input is the data-circuit, encoded as a tf.string
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    # The PQC layer returns the expected value of the readout gate, range [-1,1].
    tfq.layers.PQC(model_circuit, model_readout),
])

In [ ]:
y_train_hinge = 2.0*y_train-1.0
y_test_hinge = 2.0*y_test-1.0

In [ ]:
def hinge_accuracy(y_true, y_pred):
    y_true = tf.squeeze(y_true) > 0.0
    y_pred = tf.squeeze(y_pred) > 0.0
    result = tf.cast(y_true == y_pred, tf.float32)

    return tf.reduce_mean(result)

In [ ]:
model.compile(
    loss=tf.keras.losses.Hinge(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[hinge_accuracy])

In [ ]:
model.summary()

In [ ]:
NUM_EXAMPLES = len(x_train_tfcirc)
x_train_tfcirc_sub = x_train_tfcirc[:NUM_EXAMPLES]
y_train_hinge_sub = y_train_hinge[:NUM_EXAMPLES]

In [ ]:
qnn_history = model.fit(
      x_train_tfcirc_sub, y_train_hinge_sub,
      batch_size=32,
      epochs=100,
      validation_data=(x_test_tfcirc, y_test_hinge))

In [ ]:
train_loss, train_acc = model.evaluate(x_train_tfcirc_sub, y_train_hinge_sub)

In [ ]:
test_loss, test_acc = model.evaluate(x_test_tfcirc, y_test_hinge)

# Result: Visualization of Quantum Neural Network (QNN)

In [ ]:
train_acc = qnn_history.history['hinge_accuracy']
test_acc = qnn_history.history['val_hinge_accuracy']
train_loss = qnn_history.history['loss']
test_loss = qnn_history.history['val_loss']

epochs=range(len(train_acc)) 
plt.plot(epochs, train_acc, 'r', label= "Training Accuracy")
plt.plot(epochs, test_acc, 'b', label = "Test Accuracy")
plt.title('Training and testing accuracy')
plt.figure()

plt.plot(epochs, train_loss, 'r',label= "Training Loss")
plt.plot(epochs, test_loss, 'b', label="Test Loss")
plt.title('Training and testing loss')
plt.figure()

# References:

1. A. Abbas, D. Sutter, C. Zoufal, A. Lucchi, A. Figalli, and S. Woerner, “The power of Quantum Neural Networks,” Nature Computational Science, vol. 1, no. 6, pp. 403–409, 2021. 
2. “Deep Learning Specialization,” DeepLearning.AI, 24-Dec-2021. [Online]. Available: https://www.deeplearning.ai/program/deep-learning-specialization/. [Accessed: 03-Apr-2022]. 
3. “Deep learning,” Deep Learning. [Online]. Available: https://www.deeplearningbook.org/. [Accessed: 01-Apr-2022]. 
4. E. Farhi and H. Neven, “Classification with quantum neural networks on near term processors,” arXiv.org, 30-Aug-2018. [Online]. Available: https://arxiv.org/abs/1802.06002. [Accessed: 01-Apr-2022]. 
5. M. Broughton, G. Verdon, T. McCourt, A. J. Martinez, J. H. Yoo, S. V. Isakov, P. Massey, R. Halavati, M. Y. Niu, A. Zlokapa, E. Peters, O. Lockwood, A. Skolik, S. Jerbi, V. Dunjko, M. Leib, M. Streif, D. Von Dollen, H. Chen, S. Cao, R. Wiersema, H.-Y. Huang, J. R. McClean, R. Babbush, S. Boixo, D. Bacon, A. K. Ho, H. Neven, and M. Mohseni, “TensorFlow quantum: A software framework for Quantum Machine Learning,” arXiv.org, 26-Aug-2021. [Online]. Available: https://arxiv.org/abs/2003.02989. [Accessed: 01-Apr-2022]. 
6. “Tensorflow Quantum,” TensorFlow. [Online]. Available: https://www.tensorflow.org/quantum. [Accessed: 01-Apr-2022]. 
7. Y. LeCun, Y. Bengio, and G. Hinton, “Deep learning,” Nature, vol. 521, no. 7553, pp. 436–444, 2015. 
8. “MNIST classification &nbsp;: &nbsp; tensorflow quantum,” TensorFlow. [Online]. Available: https://www.tensorflow.org/quantum/tutorials/mnist. [Accessed: 01-Apr-2022]. 